In [9]:
import numpy as np
import pandas as pd

In [10]:
# df5 = pd.read_csv('513_164x.csv')   
df6 = pd.read_csv('327x.csv')  
# df7 = pd.read_csv('713_254x.csv')  
# df8 = pd.read_csv('813_342x.csv')  
df9 = pd.read_csv('202x.csv')  

In [11]:
df6['final'] = 0
for i in range(0,len(df6)):
    if df6['low'].iloc[i] == df9['low'].iloc[i]:
        df6['final'].iloc[i] = df6['volume'].iloc[i]

C:\Users\wzk\AppData\Local\Temp\ipykernel_30676\562478297.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['final'].iloc[i] = df6['volume'].iloc[i]


In [12]:
import matplotlib.pyplot as plt

data_1_size = len(df6)
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 25    ###### 系数 ###################
m = 100000     ###### 总资金 ###################



data_1_new = df6


for i in range(1,len(data_1_new )):
    data_1_new['high'][i] = sum(data_1_new['final'][0:(i+1)])

data_1_new['high'][0] = data_1_new['final'][0]

for i in range(0,len(data_1_new )):
    data_1_new['open'][i] = rrr * data_1_new['high'][i] / m



######################################################################################################


wp_win = data_1_new['final'] > 0
wp_lost = data_1_new['final'] < 0
wp_nothing = data_1_new['final'] == 0

### 满足条件的数量

wp_win_a = wp_win.sum()            
wp_lost_a = wp_lost.sum()
wp_nothing_a = wp_nothing.sum()


### 满足条件的数据之和

rrr_win = data_1_new[wp_win]['final'].sum()
rrr_lost = data_1_new[wp_lost]['final'].sum()




##############################################################################################
# 计算回撤数据，给到 down 列

data_1_new['down'] = 0

log = data_1_new['open'].iloc[0]

for i in range(1,len(data_1_new)):

    if data_1_new['open'].iloc[i] < log:
        data_1_new['down'].iloc[i] = data_1_new['open'].iloc[i] - log
    else:
        log = data_1_new['open'].iloc[i]


##############################################################################################
# 计算回撤面积，给到downarea列

downarea = sum(data_1_new['down'])



##############################################################################################

### 计算夏普与索提诺

data_1_new['re_real'] = 0
for i in range(1,len(data_1_new)):
    if sell == 0:
        if data_1_new['low'].iloc[i] == 0:
            data_1_new['re_real'].iloc[i] = data_1_new['re'].iloc[i] * -1
        else:
            data_1_new['re_real'].iloc[i] = data_1_new['re'].iloc[i]
    else:
        if data_1_new['low'].iloc[i] == 1:
            data_1_new['re_real'].iloc[i] = data_1_new['re'].iloc[i] * -1
        else:
            data_1_new['re_real'].iloc[i] = data_1_new['re'].iloc[i]

sharpe = round(data_1_new['re_real'][1:].mean() / data_1_new['re_real'][1:].std() * 100,4)

sortino = round(data_1_new['re_real'][1:].mean() / (data_1_new['re_real'][1:][data_1_new['re_real'][1:] < 0]).std() * 100,4)

##############################################################################################






data_1_new.to_csv('./final.csv',index = False)




s = np.argmax((np.maximum.accumulate(data_1_new['open']) - data_1_new['open'])) 
if s == 0:
    e = 0
else:
    e = np.argmax(data_1_new['open'][:s])  
maxdrawdown = data_1_new['open'][e] - data_1_new['open'][s] # 最大回撤
drawdown_days = s - e # 回撤持续周期数




start_DAY = data_1_new.index[s] #开始回撤的日期
end_DAY = data_1_new.index[e] #结束回撤的日期
start_net_value = data_1_new[data_1_new.index == start_DAY]['open'].values[0] #开始回撤的净值
end_net_value = data_1_new[data_1_new.index == end_DAY]['open'].values[0] #结束回撤的净值
fig=plt.figure(figsize=(20,11))  
plt.plot(data_1_new['eob'], data_1_new['open'])
plt.plot([start_DAY, end_DAY], [start_net_value, end_net_value], linestyle='--', color='r')

plt.xticks(range(0,data_1_size,int(data_1_size/m_size))) 

plt.legend(['All:' + str(round(data_1_new['open'].iloc[-1]*100,2)) + '%' +
            '   ' + str(m_size) + 'm'
            '   Year:'+ str(round(data_1_new['open'].iloc[-1]/m_size*100*12,2)) + '%' +
            '   CalmarY:'+ str(round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)) +
            '   WP:' + str(round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)) + '%' +
            '   RRR:' + str(round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)) + '%' + ' / ' + str(round(rrr_win/abs(rrr_lost),2)) +
            '   T/N:' + str(wp_win_a + wp_lost_a ) + ' / ' + str(wp_nothing_a) +
            '   Sharpe:' + str(sharpe) +
            '   Sortino:' + str(sortino),

            'MD:'+ str(round(maxdrawdown*100,2)) + '%' +
            '   DA:'+ str(round(downarea,4)) + '%' +
            '   MDT:' + str(drawdown_days)+
            '   Date:' + str(data_1_new['eob'].iloc[e]) + ' - ' + str(data_1_new['eob'].iloc[s])] ,

            loc='upper left',fontsize = 11)   ########### 默认是10


plt.plot(data_1_new['eob'], data_1_new['down'], color='#ec700a')   ### 桔色
plt.fill_between(data_1_new['eob'], data_1_new['down'], 0, where=(data_1_new['down']<0), facecolor='#FF0000', alpha=0.1)   
plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))                                           ### 红色 + 透明度



fig.autofmt_xdate()
plt.grid(1)
plt.savefig("final_sy.jpg")
plt.close()


fig=plt.figure(figsize=(20,10))  
plt.plot(data_1_new['eob'], data_1_new['high'])
plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))     ### 最后一个是间隔
fig.autofmt_xdate()
plt.grid(1)
plt.savefig("final_p.jpg")
plt.close()


C:\Users\wzk\AppData\Local\Temp\ipykernel_30676\4068846295.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_new['high'][i] = sum(data_1_new['final'][0:(i+1)])
C:\Users\wzk\AppData\Local\Temp\ipykernel_30676\4068846295.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_new['high'][0] = data_1_new['final'][0]
C:\Users\wzk\AppData\Local\Temp\ipykernel_30676\4068846295.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
